# Hw4 QR iteration
We use QR iteration with shifts to find eigenvalues and eigenvectors of a matrix

In [144]:
import numpy as np
import scipy.linalg as lin
from collections import deque
import copy

First, we should transform the original matrix into Hessenburg form, which means all entries under subdiagonal are zero. This can be implemented by using Householder reflector, which transform one column at one time. Dont forget to multiple Q on the right side. Since eventually we use exact eigenvalue to find eigenvectors, the Q still dont need to be constructed explictly,
$$
H = Q^*AQ
$$

In [145]:
def Hessenburg_reduction(A:np.ndarray) -> np.ndarray:
    n = A.shape[0]

    for k in range(0, n-2):
        x = A[k+1:, k] # Do not act on all rows
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x) 
        v_k = np.sign(x[0])*e + x # Choose the reflector which is further from x 
        v_k = v_k / np.linalg.norm(v_k)
        A[k+1:, k:] = A[k+1:, k:] - 2 * np.outer(v_k, v_k.T @ A[k+1:, k:])
        A[:, k+1:] = A[:, k+1:] - 2 * np.outer(A[:, k+1:] @ v_k, v_k.T) # Multiple Q on the right

    return A

Then we define the itertion step. 
1. Choose the corner entry as single shift: $ \mu = a_{n,n} $.
2. Use built-in function to compute QR factorization: $ Q_k R_k=A_k-\mu I$.
3. Update A: $ A_{k+1} = R_k Q_k + \mu T$.

In [146]:
def iteration_step(A_k:np.ndarray) -> np.ndarray:
    n = A_k.shape[0]
    shift = A_k[n-1, n-1]
    Q_k, R_k = lin.qr(A_k - shift * np.eye(n))
    A_kpls1 = R_k @ Q_k + shift * np.eye(n)

    return A_kpls1

Our final goal is the Schur factorization, so when subdiagonal entry equals zero, we can divide the matrix into two smaller part and continue the iteration separately. Here when subdiagonal is singnificantlt smaller than diagonal entries times unit roundoff(from MC 7.5.2) it is regarded zero.

In [147]:
def deflation(A:np.ndarray):
    # Since this function can have one or two outputs, we reture a list to use extend method for the queue
    n = A.shape[0]
    unit_roundoff = 1E-16
    c = 1 # A samll constant
    if n == 1:
        return [A]
    else:
        for p in range(n-1):
            if abs(A[p+1, p]) <= c * unit_roundoff * (abs(A[p, p]) + abs(A[p+1, p+1])):
                return [A[:p+1, :p+1], A[p+1:, p+1:]]
            else:
                return [A]

In [148]:
# def is_upper_triangular(matrix, threshold=1e-15):
#     lower_triangle = np.tril(matrix, k=-1)
#     return np.all(lower_triangle < threshold)

In [149]:
# def gen_orth(Q_k:np.ndarray, d_eye:int, d_q:int, n:int) -> np.ndarray:
#     d_remain = n - d_eye - d_q
#     identity_ul = np.eye(d_eye)
#     zero_up = np.zeros((d_eye, d_q))
#     zero_left = np.zeros((d_q, d_eye))
#     identity_dr = np.eye(d_remain, d_remain)
#     zero_ur = np.zeros((d_eye + d_q, d_remain))
#     zero_dl = np.zeros((d_remain, d_eye + d_q))
#     ul = np.block([
#         [identity_ul, zero_up],
#         [zero_left, Q_k]
#     ])
#     return np.block([
#         [ul, zero_ur],
#         [zero_dl, identity_dr]
#     ])

The iteration is based on queue: each step iterate the leftmost element in a queue and then delete it. The result of one iteration, maybe deflated, will be filled into the queue by order, and then go through the queue. When the length of queue after one loop equals the dimension of the matrix, which means it has been completely deflated into single diagonal entries, and it's exactly the eigenvalues, exit the outer while loop.

In [150]:
def iteration(initial:np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    H = Hessenburg_reduction(initial)
    queue = deque([H])  # Initilization
    n = initial.shape[0]
    while queue and (len(queue) != n):
        # print('length queue is', len(queue))
        next_queue = deque()

        while queue:
            current_it = queue.popleft() # Automatically delete the leftmost element
            A_kpls1 = iteration_step(current_it)
            next_queue.extend(deflation(A_kpls1))

        queue = next_queue

    return queue


Once we get very accurate eigenvalues, we can use just several inverse iterations to get the eigenvectors. We introduce a small perturbation to make the shift matrix non-singular. When the norm of $ (A-\lambda_i I)v_k $ is smaller than a threshold, reture $v_k$ as the eigenvector.

In [151]:
def inverse_iter(A:np.ndarray, lambda_i:np.complex128) -> np.ndarray:
    n = A.shape[0]
    v_k = np.zeros((n,1))
    v_k[0] = 1.0
    perturbation = 1e-6
    shift = lambda_i + perturbation
    while lin.norm((A - lambda_i * np.eye(n)) @ v_k) > 1e-12:
        omega = lin.solve(A - shift * np.eye(n), v_k)
        v_k = omega / lin.norm(omega)
        # print(v_k)

    return v_k

In [152]:
test_A = np.array([[2, 3, 2],
          [10, 3, 4],
          [3, 6, 1]], dtype=np.complex128)
dim = test_A.shape[0]
# Householder triangularization will change the initial matrix, so create a copy
A_0 = copy.deepcopy(test_A)
eigenvalues = iteration(A_0)
eigenvalues = np.squeeze(eigenvalues)  #  Get rid of the redundant dimension introduced in the return of deflation
# print(test_A)
true_eigenvalues, true_eigenvectors = lin.eig(test_A)
print("Computed eigenvalue is \n", eigenvalues)
print("True eigenvalue is \n", true_eigenvalues)
eig_vector = np.column_stack([inverse_iter(test_A, eigenvalues[i]) for i in range(0, dim)])
print('Computed eigenvector is \n', eig_vector)
print('True eigenvector is \n', true_eigenvectors)

Computed eigenvalue is 
 [11.+0.j -3.+0.j -2.+0.j]
True eigenvalue is 
 [11.+0.j -2.+0.j -3.+0.j]
Computed eigenvector is 
 [[ 3.71390676e-01+0.j -4.65661286e-17+0.j  1.82574186e-01+0.j]
 [ 7.42781353e-01+0.j  5.54700196e-01+0.j  3.65148372e-01+0.j]
 [ 5.57086015e-01+0.j -8.32050294e-01+0.j -9.12870929e-01+0.j]]
True eigenvector is 
 [[ 3.71390676e-01-0.j -1.82574186e-01+0.j -3.48372238e-16+0.j]
 [ 7.42781353e-01+0.j -3.65148372e-01+0.j -5.54700196e-01+0.j]
 [ 5.57086015e-01-0.j  9.12870929e-01+0.j  8.32050294e-01+0.j]]


In [154]:
test_B = np.array([[6, 2, 1],
          [2, 3, 1],
          [1, 1, 1]], dtype=np.complex128)
dim = test_B.shape[0]
# Householder triangularization will change the initial matrix, so create a copy
B_0 = copy.deepcopy(test_B)
eigenvalues = iteration(B_0)
eigenvalues = np.squeeze(eigenvalues)  #  Get rid of the redundant dimension introduced in the return of deflation
# print(test_A)
true_eigenvalues, true_eigenvectors = lin.eig(test_B)
print("Computed eigenvalue is \n", eigenvalues)
print("True eigenvalue is \n", true_eigenvalues)
eig_vector = np.column_stack([inverse_iter(test_B, eigenvalues[i]) for i in range(0, dim)])
print('Computed eigenvector is \n', eig_vector)
print('True eigenvector is \n', true_eigenvectors)

Computed eigenvalue is 
 [7.28799214+0.j 2.13307448+0.j 0.57893339+0.j]
True eigenvalue is 
 [7.28799214+0.j 2.13307448+0.j 0.57893339+0.j]
Computed eigenvector is 
 [[ 0.86643225+0.j  0.49742503+0.j -0.0431682 +0.j]
 [ 0.45305757+0.j -0.8195891 +0.j -0.35073145+0.j]
 [ 0.20984279+0.j -0.28432735+0.j  0.9354806 +0.j]]
True eigenvector is 
 [[ 0.86643225+0.j -0.49742503+0.j -0.0431682 +0.j]
 [ 0.45305757+0.j  0.8195891 +0.j -0.35073145+0.j]
 [ 0.20984279+0.j  0.28432735+0.j  0.9354806 +0.j]]


The experiment shows that our implement matches the built-in method in high precision.